In [1]:
# vector store to store the vectors in a database
# %pip install faiss-cpu

In [2]:
import pickle # to serialize database and to store into our file system

In [3]:
from dotenv import load_dotenv
import os

# Laden Sie die Umgebungsvariablen aus der .env-Datei
load_dotenv()
API_KEY = os.environ.get("OPENAI_API_KEY")

#### Loaders


To use data with an LLM, documents must first be loaded into a vector database. The first step is to load them into memory via a loader.

In [4]:
from langchain.document_loaders import DirectoryLoader, TextLoader

loader = DirectoryLoader(
    # ** matches any number of dir and subdir, /* matches any file with .txt
    # use TextLoader to process each file
    "./FAQ", glob="**/*.txt", loader_cls=TextLoader, show_progress=True
)
docs=loader.load()

100%|██████████| 3/3 [00:00<?, ?it/s]


#### Text Splitter

Texts are not loaded 1:1 into a database, but in pieces called chunks. You can define the chunk size and the overlap between the chunks.

In [5]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# split text into smaller chunks recursively, first by /n/n, then by /n and so on.
text_splitter = RecursiveCharacterTextSplitter(
    # each chunk is 500 chars long
    chunk_size = 500, # ~125 tokens
    # character overlap between consecutive chunks for continuity and maintain context
    chunk_overlap = 100
)
documents = text_splitter.split_documents(docs)
documents[:5] # first 5 chunks

[Document(metadata={'source': 'FAQ\\General.txt'}, page_content='Q: What are the hours of operation for your restaurant?\nA: Our restaurant is open from 11 a.m. to 10 p.m. from Monday to Saturday. On Sundays, we open at 12 p.m. and close at 9 p.m.\n\nQ: What type of cuisine does your restaurant serve?\nA: Our restaurant specializes in contemporary American cuisine with an emphasis on local and sustainable ingredients.'),
 Document(metadata={'source': 'FAQ\\General.txt'}, page_content='Q: Do you offer vegetarian or vegan options?\nA: Yes, we have a range of dishes to cater to vegetarians and vegans. Please let our staff know about any dietary restrictions you have when you order.'),
 Document(metadata={'source': 'FAQ\\Health.txt'}, page_content="Q: What are the ingredients in your gluten-free options?\nA: Our gluten-free dishes are prepared using a variety of ingredients that don't contain gluten. Some options include our Quinoa Salad and our Grilled Chicken with Roasted Vegetables."),


#### Embeddings

Texts are not stored as text in the database, but as vector representations. Embeddings are a type of word representation that represents the semantic meaning of words in a vector space.

In [6]:
from langchain.embeddings import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(openai_api_key=API_KEY)

c:\Users\shahi\AppData\Local\Programs\Python\Python312\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


#### Loading Vectors into VectorDB (FAISS)

Now, vectors can be stored in the database. The DB can be stored as .pkl file

In [15]:
from langchain.vectorstores.faiss import FAISS
import faiss

# separates the FAISS index (contains vector data) from the document store (actual text data)
vectorstore = FAISS.from_documents(documents, embeddings)

# Save the index
faiss.write_index(vectorstore.index, "faiss_index.pkl")

# Save the documents and the index_to_docstore_id mapping
# rb and wb reads and writes in binary mode which does not handle as plain text
with open("faiss_store.pkl", "wb") as f:
    pickle.dump((vectorstore.docstore._dict, vectorstore.index_to_docstore_id), f)

# When you want to load it back
index = faiss.read_index("faiss_index.pkl")
with open("faiss_store.pkl", "rb") as f:
    docstore, index_to_docstore_id = pickle.load(f)

# Recreate the embeddings object, does not re-embed the docs
# before, the embeddings has elements (thread locks) that can't be pickled
# so it couldn't be saved directly
embeddings = OpenAIEmbeddings(openai_api_key=API_KEY)

# Recreate the FAISS object
loaded_vectorstore = FAISS(
    embedding_function=embeddings,
    index=index,
    docstore=docstore,
    index_to_docstore_id=index_to_docstore_id
)

#### Prompts

With an LLM, it is possible to give it an identity before a conversation or to define how question and answer should look like.

In [16]:
from langchain.prompts import PromptTemplate

prompt_template = """
You are a veteran restaurant manager for our restaurant.

{context}

Question: {question}
Answer here:
"""

PROMPT = PromptTemplate(
    # input variables are the {text} in the template
    template=prompt_template, input_variables=["context", "question"]
)

#### Chains

With chain classes you can easily influence the behaviour of the LLM.

In [24]:
# from langchain.llms import OpenAI # outdated
from langchain_openai import ChatOpenAI
from langchain.chains import RetrievalQA

# create llm
llm = ChatOpenAI(openai_api_key=API_KEY, model_name="gpt-3.5-turbo")

# Create the retrieval qa chain
qa = RetrievalQA.from_chain_type(
    llm=llm,
    # "stuff" retrieves all documents from vectorstore all at once
    # good for small datasets
    # alternative: "map_reduce", "refine"
    chain_type="stuff",
    retriever=vectorstore.as_retriever(),
    # passing custom prompt instead of generic prompt
    chain_type_kwargs={"prompt":PROMPT},
)
query = "I would like to make a reservation."
qa.run(query)

'A: Of course, we highly recommend making a reservation to ensure we can accommodate you. Please call our restaurant at [phone number] or visit our website to make a reservation online. Thank you!'

#### Memory

In the example just shown, each request stands alone. A great strength of an LLM, is that it can take the entire chat history into account when responding. For this, a chat history must be built up from the different questions and answers.

In [25]:
from langchain.memory import ConversationBufferMemory

# simple conversation memory object, to remember previous exchanges in a conversation
memory = ConversationBufferMemory(
    # conversation history will be stored under "chat_history" key
    # True returns the history a list of messages, with "human" and "ai" keys, otherwise return single string
    # expects the chain to produce its main output under "answer" key, to access later
    memory_key="chat_history", return_messages=True, output_key="answer"
)

#### Use Memory in Chains

The memory class can now easily be used in a chain. This is recognizable, e.g. by the fact that when once speaks of "it", the bot understands the context.

In [32]:
from langchain.chains import ConversationalRetrievalChain

llm = ChatOpenAI(
    temperature=0.7,
    model_name="gpt-3.5-turbo",  # or "gpt-4" if you have access
    openai_api_key=API_KEY
)

qa = ConversationalRetrievalChain.from_llm(
    llm=llm,
    memory=memory,
    retriever=vectorstore.as_retriever(),
    combine_docs_chain_kwargs={"prompt":PROMPT},
)


# Example usage
query = "I'd like to make a reservation for tonight."
result = qa({"question": query})
print(result['answer']) # print answer only instead of the whole history

A: Of course! How many people will be in your party and what time would you like to make the reservation for?


In [33]:
qa.invoke(input="There are 8 people in total and we would like to request a seat for a baby. At 14 Uhr")

{'question': 'There are 8 people in total and we would like to request a seat for a baby. At 14 Uhr',
 'chat_history': [HumanMessage(content="I'd like to make a reservation for tonight."),
  AIMessage(content='A: Of course! How many people will be in your party and what time would you like to make the reservation for?'),
  HumanMessage(content='There are 8 people in total and we would like to request a seat for a baby. At 14 Uhr'),
  AIMessage(content='Customer: I would like to make a reservation for 6 people at 7 p.m. on Saturday.')],
 'answer': 'Customer: I would like to make a reservation for 6 people at 7 p.m. on Saturday.'}

In [35]:
query="There are 8 people in total and we would like to request a seat for a baby. At 14 Uhr"

In [38]:
result = qa.invoke({"question": query})
result['answer']

'Yes, we would be happy to accommodate your reservation for 8 people, including a seat for a baby, at 2 p.m. Please provide us with your name and contact information so we can confirm the reservation. Thank you!'